In [1]:
import os

In [2]:
import datetime
import random
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
import numpy as np

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
import math
import scipy.optimize as optim


In [5]:
import pandas as pd

In [6]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [7]:
from bokeh.embed import components


In [8]:
def do_hist(x, bins=10, width = 550, height = 550, title = None):
  hist, edges = np.histogram(x, bins = bins, density=True)
  p_hist = figure(width = width, height = height, title = title)
  p_hist.quad(top = hist, bottom=0, left=edges[:-1], right=edges[1:], alpha = .4)
  return p_hist

In [9]:
X = [random.gauss(mu = 0, sigma = 1) for x in range(100)]
X2 = [x**2 for x in X]
p = do_hist(X2)
show(p)

In [10]:
X =[random.gauss(mu = 0, sigma = 1) for x in range(100)]
X3 = [random.gauss(mu = -1000, sigma = 1) for x in range(100)]
zz = zip(X, X3)
dff = [(x[0] - x[1]) **2  for x in zz]
p  =do_hist(dff)
show(p)


In [11]:
X = [random.gauss(mu = 0, sigma = 1) for x in range(100)]
X2 = [random.gauss(mu = 1, sigma = 1) for x in range(100)]
#X is data, X2 is some type of fit
#calculate SSR from mean
m1 = np.mean(X)
ssr_mean= [(x-m1) **2 for x in X]
ssr_mean
zz = zip(X,X2)
ssr_fit = [(x[0] - x[1])**2 for x in zz]
ssr_fit

def resample_two_samples(sample1, sample2, num_iterations = 100):
    both = sample1 + sample2
    sum1 = []
    sum2 = []
    for i in range(num_iterations):
        random.shuffle(both) 
        new_1 = both[0:len(sample1)] 
        new_2 = both[len(sample1):] 
        sum1.append(np.mean(new_1)) 
        sum2.append(np.mean(new_2)) 
    return sum1, sum2

def combine_resamples(sample1, sample2, resample1, resample2):
    diff1 = [np.mean(sample1 )- x for x in resample1]
    diff2 = [np.mean(sample2)- x for x in resample2]
    both = np.array(diff1) - np.array(diff2)
    return both
r1, r2 = resample_two_samples(ssr_mean, ssr_fit)
l = combine_resamples(ssr_mean, ssr_fit, r1, r2)
sum(ssr_mean)
len([x for x in l if x > 1])


0

In [12]:
def exp_func(x, initial, ratio):
    return initial * np.power(ratio, x - 1)
X = list(range(25))
initial = 1
ratio = 2
X = list(range(1, 25))
y = [exp_func(x, initial, ratio ) + random.gauss(mu = 0, sigma = 1) for x in X]

popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(y) )
initial_hat = popt[0]
ratio_hat = popt[1]
ssr_mean =[(x-np.mean(y)) **2 for x in y]
y_fit = [exp_func(x = x, initial= initial_hat, ratio = ratio_hat) for x in X]
zz = zip(y, y_fit)
ssr_fit = [(x[0] - x[1])**2 for x in zz]
r1, r2 = resample_two_samples(ssr_mean, ssr_fit)
l = combine_resamples(ssr_mean, ssr_fit, r1, r2)
len([x for x in l if x > 0])

100

In [13]:

X = list(range(1, 25))
y = [ random.gauss(mu = 10, sigma = 1) for x in X]
popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(y) )
initial_hat = popt[0]
ratio_hat = popt[1]
ssr_mean =[(x-np.mean(y)) **1 for x in y]
y_fit = [exp_func(x = x, initial= initial_hat, ratio = ratio_hat) for x in X]
zz = zip(y, y_fit)
ssr_fit = [(x[0] - x[1])**2 for x in zz]
r1, r2 = resample_two_samples(ssr_mean, ssr_fit)
l = combine_resamples(ssr_mean, ssr_fit, r1, r2)
len([x for x in l if x > 0])


0

In [14]:
def make_trend_line_wash():
    df = pd.read_csv('data/seven_day_county.csv')
    df['date'] = pd.to_datetime(df['date'])
    last_date = df['date'].tolist()[-14]
    df_non_king_trend = df[(df['date']>=  last_date)]
    df_king = df[(df['state'] == 'Washington') & (df['county'] == 'King')]
    df_king_trend = df_king[(df_king['date']>=  last_date)]
    nums = df_king_trend['new_cases'].rolling(1).mean().tolist()
    X = range(len(nums))
    df_king_trend = df_king_trend.assign(X = X)
    labels = df_king_trend['date'].tolist()
    labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
    popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(nums) )
    print(popt[0], popt[1])
    y_hat = [exp_func(initial = popt[0], ratio = popt[1], x = x) for x in X]
    p = figure(x_axis_type = 'datetime', title = 'King {p}'.format(p = round(popt[1],2)), 
                 plot_width = 350 , plot_height = 350, y_range = None)
    p.vbar(x=labels, top=nums, line_width = 5, width = .9)
    p.line(x = labels, y = y_hat)
    model = smf.ols(formula='new_cases ~ X' , data=df_king_trend)
    res = model.fit()
    a, b1 = res.params
    y_fit2 = [a + b1 * x for x in X]
    return p
#p.line(x = labels, y = y_fit2)
#show(p)

In [15]:
def make_trend_line_king():
    df = pd.read_csv('data/non_king.csv')
    df['date'] = pd.to_datetime(df['date'])
    last_date = df['date'].tolist()[-14]
    df_non_king_trend = df[(df['date']>=  last_date)]
    nums = df_non_king_trend['cases'].rolling(1).mean().tolist()
    X = range(len(nums))
    df_non_king_trend = df_non_king_trend.assign(X = X)
    labels = df_non_king_trend['date'].tolist()
    labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
    popt, pcov = optim.curve_fit(f = exp_func, xdata =np.array(X), ydata = np.array(nums) )
    print(popt[0], popt[1])
    y_hat = [exp_func(initial = popt[0], ratio = popt[1], x = x) for x in X]
    p = figure(x_axis_type = 'datetime', title = 'Washington {p}'.format(p = round(popt[1],2)), 
                 plot_width = 350 , plot_height = 350, y_range = None)
    p.vbar(x=labels, top=nums, line_width = 5, width = .9)
    p.line(x = labels, y = y_hat)
    #print(dir(p.axis))
    p.yaxis.axis_label = 'Cases'

    
    return p
grid = gridplot([make_trend_line_king(), make_trend_line_wash()], ncols = 4)
show(grid)

551.3928641262327 1.024924623779266
133.74373333418163 1.0289924410622282


In [16]:
script, div = components(grid)
text = """RT rate for King County and Washington State. RT continues to increase (2020-07-17)."""

In [19]:
def output_graphs(script, div, text, 
                 home_page_dir = '/home/henry/projects/covid19/home_page/'):
    with open(os.path.join(home_page_dir, 'script'), 'w') as write_obj:
              write_obj.write(script)
    with open(os.path.join(home_page_dir, 'div'), 'w') as write_obj:
              write_obj.write(div)
    with open(os.path.join(home_page_dir, 'text.txt'), 'w') as write_obj:
              write_obj.write(text)
output_graphs(script, div, text = text)

In [18]:
df_pop = pd.read_csv('data/states_population.csv')
df_pop.head()
df_pop['population_2019'].dtype

dtype('int64')